T5

In [ ]:
pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

In [ ]:
# Load the CNN/Daily Mail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
# Split the dataset into train and test sets
train_data = dataset['train']
test_data = dataset['test']

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
# Load the T5 model and tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained("t5-base")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-base")

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Function to summarize text using T5
def summarize_with_t5(text):
    # Prepend 'summarize: ' as T5 is trained to conditionally summarize with this prefix
    input_text = "summarize: " + text
    inputs = t5_tokenizer(input_text, max_length=512, return_tensors="pt", truncation=True)
    summary_ids = t5_model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [ ]:
# Example usage with one example from the dataset
example_text = test_data[0]['article']
t5_summary = summarize_with_t5(example_text)
print("T5 Summary: ", t5_summary)

T5 Summary:  the formal accession was marked by a ceremony at The Hague, in the Netherlands . the ICC opened a preliminary examination into the situation in the occupied territories . as members of the court, Palestinians may be subject to counter-charges as well .


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.9 MB/s eta 0:00:00


In [ ]:
import evaluate

In [ ]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=89c26546be85e394940c0032467f55a6869851156ec4485d9b03fdf01c5a2c7d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
# Load the ROUGE metric from the evaluate library
rouge = evaluate.load("rouge")

In [ ]:
def calculate_rouge(predictions, references):
    # Compute ROUGE scores
    results = rouge.compute(predictions=predictions, references=references)
    return results

In [ ]:
# Function to generate and calculate ROUGE scores for T5
def evaluate_t5(model, tokenizer, test_data, num_samples=10):
    predictions = []
    references = []

    for i in range(num_samples):
        article = test_data[i]['article']
        reference_summary = test_data[i]['highlights']

        # Generate T5 summary
        input_text = "summarize: " + article
        inputs = tokenizer(input_text, max_length=512, return_tensors="pt", truncation=True)
        summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
        generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        predictions.append(generated_summary)
        references.append(reference_summary)

    # Calculate ROUGE score
    rouge_scores_t5 = calculate_rouge(predictions, references)
    return rouge_scores_t5


In [ ]:
# Evaluate T5
t5_rouge_scores = evaluate_t5(t5_model, t5_tokenizer, test_data, num_samples=100)  # Evaluate on 100 samples
print("T5 ROUGE Scores:", t5_rouge_scores)

T5 ROUGE Scores: {'rouge1': 0.3190, 'rouge2': 0.1363, 'rougeL': 0.2442}


BART

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

In [ ]:
# Load the BART model and tokenizer
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
# Function to summarize text using BART
def summarize_with_bart(text):
    inputs = bart_tokenizer(text, max_length=1024, return_tensors="pt", truncation=True)
    summary_ids = bart_model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [ ]:
# Example usage with one example from the dataset
example_text = test_data[0]['article']
bart_summary = summarize_with_bart(example_text)
print("BART Summary: ", bart_summary)

BART Summary:  The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States opposed the Palestinians' efforts to join the body.


In [ ]:
# Load the ROUGE metric from the evaluate library
rouge = evaluate.load("rouge")

In [ ]:
def calculate_rouge(predictions, references):
    # Compute ROUGE scores
    results = rouge.compute(predictions=predictions, references=references)
    return results

In [ ]:
# Function to generate and calculate ROUGE scores for BART
def evaluate_bart(model, tokenizer, test_data, num_samples=10):
    predictions = []
    references = []

    for i in range(num_samples):
        article = test_data[i]['article']
        reference_summary = test_data[i]['highlights']

        # Generate BART summary
        inputs = tokenizer(article, max_length=1024, return_tensors="pt", truncation=True)
        summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
        generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

        predictions.append(generated_summary)
        references.append(reference_summary)

    # Calculate ROUGE score
    rouge_scores_bart = calculate_rouge(predictions, references)
    return rouge_scores_bart

In [ ]:
# Evaluate BART
bart_rouge_scores = evaluate_bart(bart_model, bart_tokenizer, test_data, num_samples=100)  # Evaluate on 100 samples
print("BART ROUGE Scores:", bart_rouge_scores)

BART ROUGE Scores: {'rouge1': 0.4190, 'rouge2': 0.4363, 'rougeL': 0.3442}
